In [1]:
import pandas as pd
import numpy as np

In [51]:
station_df = pd.read_csv('Merged_Station.csv', index_col=0)
data_df = pd.read_csv('Merged_Data_202101_202303_v3.csv', index_col=0)

In [56]:
station_df = station_df.sort_values(by='Station_ID')
station_df.head()


,Station_ID,Lat,Long,geometry,index_right,Number,Name,Seasonal Status,Municipality,Total Docks
122,3,42.340115,-71.100619,POINT (-71.100619 42.340115),48,B32006,Colleges of the Fenway - Fenway at Avenue Loui...,Year Round,Boston,15
390,4,42.345392,-71.069616,POINT (-71.069616 42.345392),78,C32000,Tremont St at E Berkeley St,Year Round,Boston,19
558636,5,42.341814,-71.090179,POINT (-71.090179 42.341814),233,B32012,Northeastern University - North Parking Lot,Winter Storage,Boston,15
159,6,42.361257,-71.065287,POINT (-71.065287 42.361257),147,D32000,Cambridge St at Joy St,Year Round,Boston,15
1027,7,42.353391,-71.044571,POINT (-71.044571 42.353391),8,A32000,Fan Pier,Year Round,Boston,15


In [60]:
pre_df = pd.read_csv('Predicted_data_0422.csv', index_col=0)
pre_df['station_id'] = pre_df['station_id'].astype(int)
pre_df = pre_df[pre_df['station_id'].isin(station_df['Station_ID'])]
len(set(pre_df['station_id']))

434

In [62]:
import pulp

# Create the optimization model
model = pulp.LpProblem("Bike_Station_Rebalancing", pulp.LpMinimize)

# Decision variables for bikes at each station
x = {i: pulp.LpVariable(f'bike_{i}', lowBound=0, cat='Integer') for i in station_df['Station_ID']}

# Auxiliary variables for unmet and excess demands
unmet_demand = {(i, t): pulp.LpVariable(f'unmet_demand_{i}_{t}', lowBound=0) for i in station_df['Station_ID'] for t in range(24)}
excess_bikes = {(i, t): pulp.LpVariable(f'excess_bikes_{i}_{t}', lowBound=0) for i in station_df['Station_ID'] for t in range(24)}

# Costs
cost_missed = 5  # Cost per unit of unmet demand
cost_excess = 1  # Cost per unit of excess bikes

# Objective Function
model += pulp.lpSum([
    cost_missed * unmet_demand[i, t] + cost_excess * excess_bikes[i, t] 
    for i in station_df['Station_ID'] 
    for t in range(24)
])

# Constraints
for i in station_df['Station_ID']:
    for t in range(24):
        predicted_net_flow = pre_df.loc[(pre_df['station_id'] == i) & (pre_df['Split_hour'] == t), 'predicted_net_flow'].sum()
        model += unmet_demand[i, t] >= predicted_net_flow - x[i]
        model += excess_bikes[i, t] >= x[i] - predicted_net_flow
        model += x[i] <= station_df.loc[station_df['Station_ID'] == i, 'Total Docks'].values[0]  # Capacity constraint

# Solve the problem
model.solve()

# Output results
print("Status:", pulp.LpStatus[model.status])
for station in x:
    print(f"Station {station}: Allocate {x[station].varValue} bikes")


python(30672) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yuzhuoran/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/h7/cxncnmt11hsbyfjjtpglf3kr0000gn/T/d827ead61d6e4342b6922fb3fdd44774-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/h7/cxncnmt11hsbyfjjtpglf3kr0000gn/T/d827ead61d6e4342b6922fb3fdd44774-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 31253 COLUMNS
At line 105034 RHS
At line 136283 BOUNDS
At line 136718 ENDATA
Problem MODEL has 31248 rows, 21266 columns and 52080 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7574.89 - 0.11 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 5688 strengthened rows, 0 substitutions
Cgl0004I processed model has 16550 rows, 16984 columns (434 integer (0 of which binary)) and 33100 elements
Cbc0012I Integer solution of 10477.861 found by D